## Tipo Abstrato de Dado (TAD) árvore RedBlack

O código abaixo ilustra uma árvore Red-Black (Rubro Negra).

Uma árvore Red Black é uma árvore binária de busca com as 5 propriedades abaixo:

1. todo nó ou é preto ou é vermelho
2. a raiz é preta. 
3. todo nó folha é preto (e NULO)
4. se um nó é vermelho seus dois filhos são pretos 
5. todo caminho de um nó até um nó externo contém o mesmo número de nós pretos


In [1]:
#include <iostream>
#include <stdlib.h> 
using namespace std;

In [2]:
enum Cor {
    VERMELHO = 1, PRETO = 2
}

Estrutura para uma árvore binária simples.
A árvore tem ponteiro para o nó pai.

In [3]:
struct tNo {
    int chave; // pode ser modificado paraqualquer tipo de dado
    tNo *esq, *dir, *pai;
    Cor cor;
};


Função para ajustar ponteiro _pai_ da árvore binária, usada na função de exclusão.

OBS.: As duas funções abaixo não fazem parte da implementação padrão da árvore red black, mas auxiliar na programção.

In [4]:
void ajustaNoPai(tNo *no, tNo *novo){
    if (no->pai != NULL) {
        if (no->pai->esq == no)
            no->pai->esq = novo;
        else
            no->pai->dir = novo;
        if (novo != NULL)
           novo->pai = no->pai;
    } else
        novo->pai = NULL;
}

Função para ajustar o ponteiro de um nó da árvore. Usado na função de ajuste.

In [5]:
void ajustaNo(tNo *no, tNo *novo){
    if (no != NULL) {
        if (no->esq == no)
            no->esq = novo;
        else
            no->dir = novo;
        if (novo != NULL)
           novo->pai = no;
    } else
        novo->pai = NULL;
}

Rotação do nó P à esquerda.

In [6]:
tNo *rotEsquerda(tNo *p){
    tNo *q = p->dir;
    p->dir = q->esq;
    q->pai = p->pai;
    p->pai = q;
    if (q->esq != NULL)
        q->esq->pai = p;
    q->esq = p;
    return q;
}

Rotação do nó P à direita.

In [7]:
tNo* rotDireita(tNo *p){
    tNo *q = p->esq;
    p->esq = q->dir;
    q->pai = p->pai;
    p->pai = q;
    if (q->dir != NULL) 
        q->dir->pai = p;
    q->dir = p;
    return q;
}

Operação de visita do nó. Neste caso imprime a chave e a cor.

In [8]:
 void visita(tNo *no){
    if (no->cor == VERMELHO)
        cout << no->chave << ":"<< "v" <<'.' ;
    else
        cout << no->chave << ":"<< "p" <<'.' ;
}

Percurso da árvore em **PRÉ-ORDEM**.

In [9]:
void preordem(tNo *no){
    if (no != NULL){
        visita(no);
        preordem(no->esq);
        preordem(no->dir);
    }
}

Função para inicialização do nó, com alocação de memória e atribuição de valores para os componentes do nó.

De acordo com as regras de inclusão, novos nós são sempre **vermelhos**.

In [10]:
tNo *criaNo (int chave){
  tNo *n = (tNo *)malloc (sizeof (tNo));
  n->chave = chave;
  n->esq = NULL;
  n->dir = NULL;
  n->pai = NULL; 
  n->cor = VERMELHO;
  return n;
}

Função de ajuste em árvore red-black. 

Dado um nó *N* **vermelho**, o ajuste é feito caso a cor do nó pai seja **vermelho**.

A função de ajuste tem 3 casos para cada lado (3 para a esquerda e 3 para a direita):
1. tio é vermelho
2. tio é preto e o novo nó e filho direito (esq-dir)
3. tio é preto e o novo nó e filho direito (esq-dir)


In [11]:
tNo *ajustaArvore (tNo *no, tNo *raiz){
    tNo *tio = NULL, *noAsc = NULL; //no para armazenar o tio e o nó da árvore ascendente anterior 
    while (no->pai != NULL && no->pai->cor == VERMELHO){
        if (no->pai == no->pai->pai->esq){//sub arvore da esquerda
            tio = no->pai->pai->dir;
            if (tio != NULL && no->pai->pai->dir->cor == VERMELHO){ // caso 1: tio (direita do avô) é vermelho
                no->pai->cor = PRETO;
                tio->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                no = no->pai->pai;
            } else { //caso 2: tio é preto e o novo nó e filho direito
                 if (no == no->pai->dir){
                     no = no->pai;
                     noAsc = no->pai;
                     noAsc->esq = rotEsquerda(no);
                } //caso 3: tio é preto e novo nó e filho esquerdo
                no->pai->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                noAsc = no->pai->pai->pai; //arvore ascendente
                tNo *novoAvo = rotDireita(no->pai->pai);
                ajustaNo(noAsc, novoAvo);
                if (novoAvo->pai == NULL)
                    raiz = novoAvo;            
            }
        } 
        else { //sub arvore da direita. Mesmo princípio do código acima, porém com ponteiros (esq-dir) "trocados"
            tio = no->pai->pai->esq;
            if (tio != NULL && no->pai->pai->dir->cor == VERMELHO){ //caso 2:
                no->pai->cor = PRETO;
                tio->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                no = no->pai->pai;
            } else {
                 if (no == no->pai->esq){
                     no = no->pai;
                     noAsc = no->pai;
                     noAsc->dir = rotDireita(no);
                }
                no->pai->cor = PRETO;
                no->pai->pai->cor = VERMELHO;
                noAsc = no->pai->pai->pai; //arvore ascendente
                tNo *novoAvo = rotEsquerda(no->pai->pai);
                ajustaNo(noAsc, novoAvo);
                if (novoAvo->pai == NULL)
                    raiz = novoAvo;            
            }           
        } 
    }
    raiz->cor = PRETO;
    return raiz;
}

Funcão iterativa para inclusão de novo nó na árvore, dado um nó raiz.

Após a inclusão, chama a função de ajuste.

In [12]:
tNo* inclui (tNo *no, int c, tNo *r){
    tNo *novoNo;
    if (no == NULL) { //inclui na raiz
        novoNo = criaNo(c);
        novoNo->cor = PRETO;
        return novoNo;
    }
    tNo *pai, *raiz = r;
    while (no != NULL)  {
        pai = no;
        if ( c < no->chave)
            no = no->esq;
        else
            no = no->dir;
    }
    novoNo = criaNo(c);
    if (c < pai->chave)
        pai->esq = novoNo;
    else
        pai->dir = novoNo;
    novoNo->pai = pai;
    
    raiz = ajustaArvore(novoNo, raiz);
    return raiz;        
}

Função que lê um token, separado por espaço, e converte para um número inteiro.

In [13]:
int token_to_num(const char *str, int *indice){    
    char token[100];
    int i = 0;
    while (str[*indice] != '\0' && str[*indice] != ' '){
        token[i] = str[*indice];
        i++;
        (*indice)++;
    }
    token[i] = '\0';
    (*indice)++;    
    return atoi(token);
}

Função que monta uma árvore binária recebendo como entrada uma árvore com parênteses aninhados. Não há suporte a erros de entrada, por isso a árvore passada como parâmetro deve estar correta.

In [14]:
tNo* montaarvore(const char *str){
    tNo *raiz = NULL;
    int i = 0, v =0;
    raiz = inclui(NULL, token_to_num(str,&i), NULL);
    while (str[i]!='\0'){
        raiz = inclui (raiz, token_to_num(str, &i),raiz);                
    }
    return raiz;        
}

Percurso da árvore em **ORDEM**.

In [15]:
void emordem(tNo *no){    
    if (no !=NULL){
        emordem(no->esq);
        visita(no);
        emordem(no->dir);
    }
}

Percurso da árvore em **PÓS-ORDEM**.

In [16]:
void posordem(tNo * no){
    if (no !=NULL){    
        posordem(no->esq);        
        posordem(no->dir);
        visita(no);
    }
}

Conta o número de nós da árvore.

In [17]:
int contaNos(tNo *no){
    if (no!=NULL) {
        return contaNos(no->esq) + contaNos(no->dir) + 1;
    } else
        return 0;
}

Calcula a altura da árvore. Retorna -1 para considerar a altura da raiz como 0.

In [18]:
int altura (tNo *p) {
    int he, hd;
    if (p == NULL) return -1;
    he = altura (p->esq);
    hd = altura (p->dir);
    if (he > hd)
        return he+1;
    else
        return hd+1;    
}

Busca em árvore binária. Imprime a chave do nó que está sendo visitado para mostrar o caminho percorrido.

In [19]:
tNo *busca(tNo *no, int chave){
    if (no == NULL) return NULL;
    if (no->chave == chave) return no;
    if (chave < no->chave)
        return busca (no->esq, chave);
    else
        return busca (no->dir, chave);
}

Função auxiliar para imprimir a chamada de várias operações.

In [20]:
void imprime(const char *str, tNo *no){
    cout << "||" << str << " ";
    if (no!=NULL) cout << "["<< no->chave<<"]";
    else cout << "não";
    cout << " encontrado\n";
}

Função que retorna o nó mínimo de uma sub-árvore. O nó mínimo de uma subárvore é o nó com a menor chave.

In [21]:
tNo *min(tNo *no){
    if (no->esq == NULL) return no;
    else
        return min(no->esq);
}

Função que retorna o nó sucessor de um determinado nó. A chave do nó deverá ser o valor imediatamente superior, em ordem crescente. 

As funções do antecessor e máximo possuem ideia semelhante as do sucessor e mínimo, porém para o lado contrário da árvore.

In [22]:
tNo *sucessor (tNo *no){
    tNo *s = NULL;
    if (no->dir != NULL) return min (no->dir);
    else {
        s = no->pai;
        while (s != NULL && no == s->dir) {
            no = s;
            s = s->pai;
        }        
    }
    return s;
}

In [23]:
void iniciaprograma(){
    tNo *raiz = NULL, *no= NULL;
   // char entrada[] = "20 7 12 30 22\0";
  //  char entrada[] = "20 7 12 30 22\0";
     char entrada[] = "20 22 21\0";
    
    raiz=montaarvore(entrada);    
    cout << "\nPercurso em pré-ordem: ";
    preordem(raiz);
    cout << "\nTotal de nós: " << contaNos(raiz);
    cout << "\nAltura da árvore: " << altura(raiz) << "\n";
    cout << "\n";
    
    imprime("busca", busca (raiz,15));
      
}

In [24]:
iniciaprograma();

&21&
Percurso em pré-ordem: 21:p.20:v.22:v.
Total de nós: 3
Altura da árvore: 1

||busca não encontrado
